In [1]:
#Esto me sirve para el correcto funcionamiento de las funciones importadas en este notebook
%load_ext autoreload
%autoreload 2

# Creando un clon del IMDB Top 250 con Pandas

## Contexto

La base de datos de películas en Internet (IMDB) mantiene una lista llamada **IMDB Top 250**, que es una clasificación de las 250 mejores películas según un determinado criterio de puntuación. Todas las películas en esta lista son estrenos teatrales no documentales con una duración mínima de 45 minutos y más de 250,000 calificaciones.

Como primer acercamiento al mundo de los sistemas de recomendación, me voy a proponer crear un "clon" del top 250 películas de IMDB. Este proceso me servirá para entender las bases de los sistemas de recomendación de una forma simple, para luego ir complejizando los mismos.

Los datos usados en este notebook pertenecen a **"The Movies Dataset"**, el cual fue sacado de [Kaggle](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?resource=download&select=movies_metadata.csv).

![Captura top 250 IMDB](../data/images/captura_top250_IMDB.png)

Podemos ver que esta lista puede considerarse como el sistema de recomendación más simple. No toma en cuenta los gustos de un usuario en particular, ni intenta deducir similitudes entre diferentes películas. Simplemente calcula una puntuación para cada película basada en un criterio predefinido y genera una lista ordenada de películas basada en esa puntuación.

## Cargando los datos

Cargo las librerías que voy a usar:

In [3]:
from utils.paths import crear_funcion_directorio
import pandas as pd

Leo el CSV que contiene los metadatos de las películas:

In [4]:
#Creo un acceso directo a la carpeta de data
data_dir = crear_funcion_directorio("data")

#Cargo el dataset
movies_metadata_df = pd.read_csv(data_dir("raw", "movies_metadata.csv"), low_memory=False)

Ahora paso a ver la estructura de filas y columnas del dataframe:

In [6]:
movies_metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Podemos ver que es un dataframe con gran cantidad de features que servirán para generar el sistema de recomendación. Entre los más interesantes están **'vote_count'** y 
**'vote_average'** que miden la cantidad de votos y promedio de puntuación que recibió una película en partícular respectivamente, estos features son los principales para componer la lista.


## La métrica usada y los prerrequisitos

Para generar esta lista se necesita **una métrica para posicionar las películas** y **prerrequisitos** que deben cumplir las mismas para ser incluidas en la lista en primera instancia.

**La métrica** es un valor númerico que define si una película es mejor que otra, si una película tiene un mayor valor en dicha métrica entonces será considerada "mejor" que otra. Elegir una métrica es arbitrario y lo primero que se podría venir a la mente es tomar el rating que recibió una película, pero dicha métrica en sí tiene muchos problemas, para visualizar dichos problemas podemos considerar los siguientes puntos:

1. Tenemos una película X con solo 10 votos y todos obtuvieron la máxima puntuación
2. Tenemos otra película Y con más de 10000 votos y su promedio en puntuación es de 9.2
3. Al basar nuestra métrica en el rating nos daría que la película X es mejor que la Y, pero fácilmente podría ocurrir que la película X
es una "película de nicho" que solo le gusta a un grupo selecto de personas y que no representa a la población general

Con estos puntos logré dejar clara la importancia de **elegir una métrica robusta** y ciertos prerrequisitos que las películas deben cumplir antes de entrar a la lista. Con esto visto entonces deberíamos buscar una métrica que tome en cuenta la cantidad de votos y el rating de la película, afortunadamente dicha métrica existe y viene proporcionada por IMDB. A continuación una imagen de la fórmula que describe la métrica y sus parámetros explicados:

![Formula métrica IMDB](../data/images/formula_descripcion_IMDB.png)

Podemos ver que en nuestro dataframe ya existen los valores de **v** y **R** para cada película, los cuales vienen dados por las columnas **'vote_count'** y 
**'vote_average'** respectivamente, luego calcular **C** es trivial y **m** representa el prerrequisito que nosotros queremos definir.

Al igual que con la métrica **elegir un valor de m es arbitrario**, pero hay que saber que a mayor valor de **m** mayor será el enfásis que se le dá a la popularidad de la película y por lo tanto tendremos una lista más pequeña. 

Para este caso me gustaría considerar a las películas que recaudaron una cantidad de votos **mayor o igual al 85% de las películas en el dataset**, es decir que tomaré el percentil 85 de **'vote_count'**. Voy a llevarlo a Python:

In [9]:
m = movies_metadata_df['vote_count'].quantile(0.85)

# Lo imprimo
m

np.float64(82.0)

Es decir que **solo el 15%** de las películas en el dataframe han logrado recaudar más de 82 votos.

Ahora paso a generar un nuevo dataframe **'movies_lista'** que contenga solo las películas que tengan como mínimo el valor de m en la columna **'vote_count'**:

In [10]:
# Genero el nuevo dataframe
movies_lista = movies_metadata_df.copy().loc[movies_metadata_df['vote_count'] >= m]

# Miro sus dimensiones
movies_lista.shape

(6832, 24)

Pasamos de tener una 45000 películas a apróximadamente 7000, eso muestra lo estricto que es nuestro **m**.

Ahora consigo el valor de **C**:

In [12]:
# Calculo el valor de C, notar que lo tomo desde el dataframe original
C = movies_metadata_df['vote_average'].mean()

# Lo imprimo
C

np.float64(5.618207215134185)

Podemos ver también lo estricto de los ratings en IMDB, siendo que el promedio de estas películas **es apróximadamente 5.62**. 

Ahora defino una función que calcule la métrica para una película dada:

In [13]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Retorno la métrica basada en la fórmula de IMDB
    return (v/(v+m) * R) + (m/(m+v) * C)

Y ahora generamos una nueva feature llamada **'score'** en el dataframe **'movies_lista'** que contenga la métrica calculada para cada película, lo que nos servirá para finalmente obtener la lista que buscábamos en un principio:

In [16]:
# Aplico la función
movies_lista['score'] = movies_lista.apply(weighted_rating, axis=1)

# Ordeno el dataframe y lo muestro por pantalla
movies_lista = movies_lista.sort_values('score', ascending=False)
movies_lista[['title', 'vote_count', 'vote_average', 'score', 'runtime']].head(20)

,title,vote_count,vote_average,score,runtime
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.715738,190.0
314,The Shawshank Redemption,8358.0,8.5,8.472002,142.0
834,The Godfather,6024.0,8.5,8.461299,175.0
40251,Your Name.,1030.0,8.5,8.287494,106.0
12481,The Dark Knight,12269.0,8.3,8.282195,152.0
2843,Fight Club,9678.0,8.3,8.277469,139.0
292,Pulp Fiction,8670.0,8.3,8.274874,154.0
522,Schindler's List,4436.0,8.3,8.251326,195.0
23673,Whiplash,4376.0,8.3,8.250671,105.0
5481,Spirited Away,3968.0,8.3,8.245702,125.0


Y aquí se logró obtener una lista bastante parecida a la mostrada anteriormente, logrando así el objetivo que se propuso. Podemos ver que la película en el puesto 1 tiene una cantidad de votos notablemente menor que el resto en la lista, pero esto puede cambiar ajustando el valor de **m** para que sea más estricto.